In [9]:
import pandas as pd

file_path = "internet_service_churn.csv"

df = pd.read_csv(file_path)
print("Uploaded")

Uploaded


In [13]:
import contextlib
import joblib
from tqdm.auto import tqdm
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report


df_clean = df.copy()

# Видалення ID
if "id" in df_clean.columns:
    df_clean = df_clean.drop("id", axis=1)

df_clean["reamining_contract"] = df_clean["reamining_contract"].fillna(0)
df_clean["download_avg"] = df_clean["download_avg"].fillna(df_clean["download_avg"].mean())
df_clean["upload_avg"] = df_clean["upload_avg"].fillna(df_clean["upload_avg"].mean())

X = df_clean.drop("churn", axis=1)
y = df_clean["churn"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("Data cleaned and scaled!")

# Вивід прогресс бара
@contextlib.contextmanager
def tqdm_joblib(tqdm_object):
    class TqdmBatchCompletionCallback(joblib.parallel.BatchCompletionCallBack):
        def __call__(self, *args, **kwargs):
            tqdm_object.update(n=self.batch_size)
            return super().__call__(*args, **kwargs)

    old_batch_callback = joblib.parallel.BatchCompletionCallBack
    joblib.parallel.BatchCompletionCallBack = TqdmBatchCompletionCallback
    try:
        yield tqdm_object
    finally:
        joblib.parallel.BatchCompletionCallBack = old_batch_callback
        tqdm_object.close()

# Параметри для перебору
param_grid = {
    "C": [200],
    "gamma": ["scale"],
    "kernel": ["rbf"]
}

# n_jobs=-1 для швидкості
grid = GridSearchCV(SVC(probability=True), param_grid, cv=3, n_jobs=-1, verbose=0)

total_tasks = len(param_grid["C"]) * len(param_grid["gamma"]) * len(param_grid["kernel"]) * 3

print(f"Start training! \nTotal tasks: {total_tasks}")

with tqdm_joblib(tqdm(desc="Training SVM", total=total_tasks)) as progress_bar:
    grid.fit(X_train_scaled, y_train)

print("\nTraining completed")
print(f"Best settings: {grid.best_params_}")
print(f"Accuracy: {grid.best_score_:.2%}")

# Збереження
joblib.dump(grid.best_estimator_, "churn_svm_model.pkl")
joblib.dump(scaler, "churn_scaler.pkl")
print("Files saved")

Data cleaned and scaled!
Start training! 
Total tasks: 3


Training SVM: 100%|██████████████████████████████████████████████████████████████████| 3/3 [1:36:42<00:00, 1934.04s/it]


Training completed
Best settings: {'C': 200, 'gamma': 'scale', 'kernel': 'rbf'}
Accuracy: 91.71%
Files saved
